In [ ]:
import csv
import random
from autogluon.tabular import TabularDataset, TabularPredictor
property_label = "P161"
orientation = "subjects"
limit = "full"

In [ ]:
# load instance dataset
instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.{orientation}.{property_label}.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in instances:
        instances[line[0]] = [line[1]]
    else:
        instances[line[0]].append(line[1])

In [ ]:
# get top 10 classes from data
all_classes = set()
instance_count = {}
with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.1250.tsv",encoding="utf-8") as file:
    tsv_file = csv.reader(file, delimiter="\t")
    next(tsv_file)
    for line in tsv_file:
        all_classes.add(line[0])
for qnode in all_classes:
    instance_list = instances[qnode]
    for this_instance in instance_list:
        if this_instance in instance_count:
            instance_count[this_instance]+=1
        else:
            instance_count[this_instance]=1
print({k: v for k, v in sorted(instance_count.items(), key=lambda item: item[1], reverse=True)})

In [ ]:
# filter dataset for top 10 classes
classes_to_use = set()
lines_to_write = []
top_10_classes = ["Q11424","Q5398426","Q506240","Q7777570","Q21191270","Q20667187","Q229390","Q43099869","Q15416","Q61220733"]
with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.1250.tsv",encoding="utf-8") as file:
    tsv_file = csv.reader(file, delimiter="\t")
    next(tsv_file)
    for line in tsv_file:
        for this_instance in instances[line[0]]:
            if this_instance in top_10_classes:
                lines_to_write.append(line)
                classes_to_use.add(line[0])
print(len(classes_to_use))
# with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.top10classes.tsv", 'wt',encoding="utf-8") as out_file:
#     tsv_writer = csv.writer(out_file, delimiter='\t')
#     tsv_writer.writerow(["node","nodeLabel","prop","nodeConnection","connectionLabel","nodeOrientation"])
#     tsv_writer.writerows(lines_to_write)

In [ ]:
lines_to_write = []
with open("../embeddings/profile_graph_embeddings.TransE.tsv") as file:
    tsv_file = csv.reader(file, delimiter=" ")
    next(tsv_file)
    for line in tsv_file:
        if line[0] in classes_to_use:
            instances_to_use = []
            for this_instance in instances[line[0]]:
                if this_instance in top_10_classes:
                    instances_to_use.append(this_instance)
            newline = line
            newline.append(random.choice(instances_to_use))
            lines_to_write.append(newline)
with open(f"../output/instance_prediction_datasets/{property_label}.{orientation}.embeddings_data.{limit}.tsv", 'wt',encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Qnode","pos1","pos2","pos3","pos4","pos5","pos6","pos7","pos8","pos9","pos10","pos11","pos12","pos13","pos14","pos15","pos16","pos17","pos18","pos19","pos20","pos21","pos22","pos23","pos24","pos25","pos26","pos27","pos28","pos29","pos30","pos31","pos32","pos33","pos34","pos35","pos36","pos37","pos38","pos39","pos40","pos41","pos42","pos43","pos44","pos45","pos46","pos47","pos48","pos49","pos50","pos51","pos52","pos53","pos54","pos55","pos56","pos57","pos58","pos59","pos60","pos61","pos62","pos63","pos64","pos65","pos66","pos67","pos68","pos69","pos70","pos71","pos72","pos73","pos74","pos75","pos76","pos77","pos78","pos79","pos80","pos81","pos82","pos83","pos84","pos85","pos86","pos87","pos88","pos89","pos90","pos91","pos92","pos93","pos94","pos95","pos96","pos97","pos98","pos99","pos100","Yval"])
    tsv_writer.writerows(lines_to_write)

In [ ]:
# apply SMOTE
# TODO

In [ ]:
# fit autogluon models
all_data = TabularDataset(f'../output/instance_prediction_datasets/{property_label}.{orientation}.embeddings_data.{limit}.tsv')
train_data = all_data[:708]
test_data = all_data[708:]
save_path = f'../embeddings/models/agModels.{property_label}.{orientation}.{limit}'
label = 'Yval'
print("Summary of class variable: \n", train_data[label].describe())
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

In [ ]:
# build test data and evaluate models
y_test = test_data[label]
test_data_nolab = test_data.drop(columns=[label])
print(predictor.get_model_best())
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
predictor.leaderboard(test_data, silent=True)